<div align="right">Massimo Nocentini
<br>June 7, 2016: theoretical intro
<br>May 19, 2016: BRGC
</div>

<div align="center">
<b>Abstract</b><br>
This document collect some theory and arguments about <i>Gray codes</i>.
</div>

In [39]:
%run ../python-libs/inpututils.py
%run ../python-libs/graycodes.py
%run ../python-libs/bits.py

# Combinatorial Generation

The theory described here is given by [Frank Ruskey][fr] and adapted from his [Combinatorial Generation][cg] book. We use the following arguments to implement some Python functions, available [here][libs:gray]. In this introduction we report some concepts and definitions from Ruskey that we believe interesting and necessary for what follows, starting with the meaning of *Combinatorial Generation*:

>*Let's look at all the possibilities.* This phrase sums up the outlook of this book. In
computer science, mathematics, and in other fields it is often necessary to examine all of
a finite number of possibilities in order to solve a problem or gain insight into the solution
of a problem. These possibilities often have an underlying combinatorial structure which
can be exploited in order to obtain an efficient algorithm for generating some appropriate
representation of them.

About *complexity*:

>Not much attention has been paid to the model of computation in the area of combinatorial
generation. For the most part authors use the random access machine model. This is more
or less like counting operations in a Pascal program, with the understanding that integers
can be arbitrarily large and the assumption that standard arithmetic operations can be
done in constant time. For large integers this is an unrealistic assumption. It is generally
the case that integers are small in generation algorithms, but that they are large in ranking
and unranking algorithms. We will therefore measure ranking and unranking algorithms in
terms of the number of arithmetic operations used.

About *analysis*:

>There are two terms that are used throughout the book. We strive to develop algorithms
that have these properties. 
   - **CAT** algorithms: the holy grail of generating combinatorial objects is to find an algorithm
   that runs in Constant Amortized Time. This means that the amount of computation, after
   a small amount of preprocessing, is proportional to the number of objects that are listed.
   *We do not count the time to actually output or process the objects; we are only concerned
   with the amount of data structure change that occurs as the objects are being generated*. 
   Suppose that the input to our generation algorithm is n, and that this will produce $N$
   objects, each of "size" $n$. A CAT algorithm has running time $\mathcal{O}(N)$. In the typical 
   application of combinatorial generation, each object, as it is produced, is processed somehow.
   If this processing takes time $\mathcal{O}(n)$, then having a CAT algorithm has no advantage over
   having one with running time $\mathcal{O}(nN)$, since the total running time is $\mathcal{O}(nN)$ 
   in either case.
   - **BEST** algorithms: this means *Backtracking Ensuring Success at Terminals*. In other
   words, the algorithm is of the backtracking type, but every leaf of the backtracking tree is
   an object of the desired type; it is a *success*.

*Don't count the output* principle:

>There are are many papers about generating combinatorial objects that describe an
$\mathcal{O}(nN)$ running time as *optimal*, because this is the amount of output produced by the
algorithm — each object has size $n$ and there are $N$ objects in total. This is a misguided
notion that has its roots in the way lower bounds are discussed in the traditional introduction
to complexity theory course. There the *trivial lower bound* says that the amount of output
gives a lower bound on the amount of computation — of course it says the same thing in
our setting, *but the amount of output is the wrong thing to measure*.
>>**Don't Count the Output Principle**: In combinatorial generation it is the
amount of data structure change that should be measured in determining the complexity 
of an algorithm; the time required to output (and process) each object should be ignored.

>The *trivial lower bound* for combinatorial generation is $\mathcal{Θ}(N )$; it is independent of $n$.
Now back to our typical application,
where processing each object took time $\mathcal{O}(n)$ even though our generation algorithm was
CAT. Wouldn't it be nice if the $\mathcal{O}(nN)$ could be brought down to $\mathcal{O}(N)$? 
This is frequently possible! If you go back and observe how successive objects are processed, *it is often the
case that the amount of processing required is proportional to the amount of change that
successive objects undergo.*

A last concept, *ordering*:

>*Lexicographic* order is based on the familiar idea as the ordering of words in dictionaries.
The only requirement is that *the letters that make up the alphabet of the language be
ordered*. In the definitions below we use $\prec$ to denote the assumed underlying ordering of
the symbols of the alphabet and $<$ to denote orderings of strings (and, of course, integers). 
We remark again that in most instances the alphabet is the set of natural numbers under the usual numeric ordering
$0 \prec 1 \prec 2 \prec \ldots$.

We report definitions of three orderings:

   - *lexicographic* (or *lex*) order, denoted by $<_{l}$: 
   $a_1 a_2 \ldots a_n <_l b_1 b_2 \ldots b_m$ if either
      1. for some $k$, $a_k \prec b_k$ and $a_i = b_i$ for $i\in\lbrace 1, 2, \ldots , k − 1\rbrace$, or
      2. $n < m$ and $a_i = b_i$ for $i \in\lbrace 1, 2, \ldots , n\rbrace$.
   - *reverse lexicographic* (or *relex*) order, denoted by $<_{r}$: $a_1 a_2 \ldots a_n <_r b_1 b_2 \ldots b_m$
   if $b_1 b_2 \ldots b_m <_l a_1 a_2 \ldots a_n$ in lex order.
   - *co-lexicographic* (or *colex*) order, denoted by $<_{c}$: $a_1 a_2 \ldots a_n <_c b_1 b_2 \ldots b_m$
   if $a_n \ldots a_2 a_1 <_l b_m \ldots b_2 b_1$ in lex order.

[fr]:http://webhome.cs.uvic.ca/~ruskey/
[cg]:http://www.1stworks.com/ref/RuskeyCombGen.pdf
[libs:gray]:https://github.com/massimo-nocentini/competitive-programming/blob/master/python-libs/graycodes.py

# Binary Reflected Gray Codes

Here we introduce combinatorial Gray codes, which are lists of combinatorial
objects such that are pairwise *close*. Quoting Ruskey:

>Suppose that we wish to list all $2^n$ bitstrings of length $n$, in such a way that each bitstring
differs from its predecessor by a single bit. Such lists are called *binary Gray codes*, named
after Frank Gray who used them in a patent he obtained for "pulse code communication".

Using *reflection*, it yields a recursive definition:

>There is a very simple recursive algorithm for generating one particular Gray code, known
as the *binary reflected Gray code (BRGC)*.
The binary reflected Gray code on *n* elements, $\textbf{G}_n$, is defined recursively by letting
$\textbf{G}_0$ be the single element sequence containing the empty string $\epsilon$, 
and letting $\textbf{G}_n$ be the list obtained by prefixing each element of $\textbf{G}_{n−1}$ 
with a 0 and then prefixing each element of $\textbf{G}_{n−1}$ *in reversed order* with a 1. For example: 
$\textbf{G}_1 = (0,1), \textbf{G}_2 = (00, 01, 11, 10), \textbf{G}_3 = (000, 001, 011, 010, 110, 111, 101, 100)$.
The $n$-dimensional hypercube, or $n$-cube, $Q_n$ , may be considered as the graph whose
vertices consist of all bitstrings of length $n$ and where two vertices are joined by an edge
if they differ by a single bit. The problem of finding a Gray code corresponds to finding a
Hamilton path in $Q_n$: observe that the binary reflected Gray
code actually defines a Hamiltonian cycle on the $n$-cube since the first and last bitstrings
only differ by a single bit.

There is an interesting connection between the Towers of Hanoi problem, the binary
reflected Gray code, and ordinary counting in binary:

>Now suppose that the disks are numbered $0, 1, \ldots , n-1$ where 0 is the smallest disk and
$n-1$ is the largest disk. Let $T_n$ denote the sequence of disks that are moved in solving the
$n$ disk Towers of Hanoi problem. Looking at the following table, $T_3 = 0, 1, 0, 2, 0, 1, 0$. 
It turns out that the number at position k in T_n, zero-based, 
is the index of the bit that changes in the bitstring of $\textbf{G}_n$ in position $k$, 
and that this is equal to the position $k$ 
of the rightmost 0, counting from the right, in the bitstring when counting in binary.
The sequence $T_n$ is called the transition sequence of the BRGC. Every Hamilton path on
the $n$-cube has a transition sequence and these paths are often specified by their transition
sequences.
$$
\begin{array}{ccc}
\text{binary} & \text{Gray code} & T_n\\
\hline
000 & 000 & 0 \\
001 & 001 & 1 \\
010 & 011 & 0 \\
011 & 010 & 2 \\
100 & 110 & 0 \\
101 & 111 & 1 \\
110 & 101 & 0 \\
111 & 100 &  \\
\end{array}
$$

We will first derive two functions, for *ranking* and *unranking* gray codes and, lately, a function to efficiently compute the position of the rightmost 0 when counting in binary.

## *Ranking* Gray codes

As a notational convenience we will now assume that the Gray code
bitstrings are indexed $g_{n} \ldots g_1$, in contrast to the usual 0-based indexing.

Let $r$, standing for *rank*, be a function defined inductively as follows:
$$
    r()=0 \\
    r(0 g_{n-1} \ldots g_{1} ) = r(g_{n-1} \ldots g_{1} ) \\
    r(1 g_{n-1} \ldots g_{1} ) = 2^{n-1} + \bar{r}(g_{n-1} \ldots g_{1} )
$$
where auxiliary function $\bar{r}$ is defined as:
$$
    \bar{r}()=0 \\
    \bar{r}(0 g_{n-1} \ldots g_{1} ) = 2^{n-1} + \bar{r}(g_{n-1} \ldots g_{1} )  \\
    \bar{r}(1 g_{n-1} \ldots g_{1} ) = r(g_{n-1} \ldots g_{1} )
$$

In Ruskey words, slightly modified:

>Let $\textbf{g}=g_{n}\ldots g_{1}$ be a Gray code and assume $r(g_{n}\ldots g_{1})=(b_{n-1}\ldots b_{0})_{2}$. This mutual recursive definition may be interpreted as setting $b_{n−1}$ to 0 or 1, depending upon whether
the term $2^{n−1}$ in the binary expansion is present or not. The repeated application of these
recurrence relations to a bitstring may then be thought of as sweeping functions $r$ or $\bar{r}$ from left-to-right as illustrated in the example below, where $\textbf{g}=g_{n}\ldots g_{1} = 111010101$:

$$
\begin{array}{cccccccccc}
       & g_{9} & g_{8} & g_{7} & g_{6} & g_{5} & g_{4} & g_{3} & g_{2} & g_{1} \\
     r & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 \\
     1 & \bar{r} & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 \\
     1 & 0 & r & 1 & 0 & 1 & 0 & 1 & 0 & 1 \\
     1 & 0 & 1 & \bar{r} & 0 & 1 & 0 & 1 & 0 & 1 \\
     1 & 0 & 1 & 1 & \bar{r} & 1 & 0 & 1 & 0 & 1 \\
     1 & 0 & 1 & 1 & 0 & r & 0 & 1 & 0 & 1 \\
     1 & 0 & 1 & 1 & 0 & 0 & r & 1 & 0 & 1 \\
     1 & 0 & 1 & 1 & 0 & 0 & 1 & \bar{r} & 0 & 1 \\
     1 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & \bar{r} & 1 \\
     1 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & r \\
     b_{8} & b_{7} & b_{6} & b_{5} & b_{4} & b_{3} & b_{2} & b_{1} & b_{0} \\
\end{array}
$$

>With $g_{n+1}$ = 0, note that an $r$ always has a 0 to its left and a $\bar{r}$ has a 1 to its left. Thus
for each sub-sequence $b_{i}\,\sigma\,g_{i}$, where $\sigma\in\lbrace r, \bar{r}\rbrace$, there are *four* possibilities $\lbrace 0\,r\,0, 0\,r\,1, 1\,\bar{r}\,0, 1\,\bar{r}\,1\rbrace$. The resulting values of $b_{i−1}$ are $\lbrace 0, 1, 1, 0 \rbrace$ in those four cases, respectively:

$$
\begin{array}{ccc}
    b_{i} & \sigma & g_{\hat{i}} \\
     & \downarrow & \\
    b_{i} & b_{i-1} & \sigma' \\
\end{array}
$$

>where $\downarrow$ is the application of function $\sigma \in \lbrace r, \bar{r}\rbrace$, which sets $b_{i-1}$ and function $\sigma'$ according to the mutual definition. From this we observe that $b_{i−1} = b_i \oplus g_{\hat{i}}$ , where $\oplus$ denotes exclusive-or: the bit below an $r$ or $\bar{r}$ is the exclusive-or of the bits left and right of the $r$ or $\bar{r}$. Therefore the following holds:

$$ r(g_{n}\ldots g_{1})=(b_{n-1}\ldots b_{0})_{2} \rightarrow b_{i−1} = b_i \oplus g_{\hat{i}} $$

>where $g_{\hat{i}} = g_{i-1}$.
 
We can use the above argument to produce the Gray code with generic symbol $g_{i-1}$, zero-based indexing, in rank or position $k$ (named $b$ above) using the relation $k_i \oplus k_{i−1} =  g_{i-1}$ and visualized as:

$$
\begin{array}{ccccc}
    b_{i} & & b_{i-1} & & b_{i-2} \\
    \oplus & \searrow & \oplus & \searrow & \oplus \\
    b_{i+1} & & b_{i} & & b_{i-1} \\
    = & & = & & = \\
    g_{i} & & g_{i-1} & & g_{i-2} \\
\end{array}
$$

observe that the former and latter rows stay steal, only the middle is shifted to the right.

### Indirect generation

Python function `graycode_unrank` that implements arguments above is pretty elegant; however, this generation is *indirect* since it does not provide the *position* of the changing bit among adjacent codes:

In [26]:
python_code(graycode_unrank)

```python
def graycode_unrank(k): 
    """
    Return the *graycode* in position `k` respect binary reflected construction.
    
    Of course, argument `k` should be a *non-negative* integer.

    Examples
    ========

    >>> bin(graycode_unrank(0b101100110))
    '0b111010101'

    """
    g = k ^ (k >> 1)
    return g

```

The following is an iterable of Gray codes we saw in the example above, namely all of them of length 3, pretty-printed:

In [27]:
print('\n'.join(list(binary_reflected_graycodes(3, justified=True))))

0b000
0b001
0b011
0b010
0b110
0b111
0b101
0b100


Moreover, we have implemented an *operator* which allows us to process an iterable of codes pairwise, possibly plugging-in custom behavior via `lambda` expressions. Previous table with the position of the changing-bit can be rebuilt as follows:

In [28]:
from itertools import count

example_graycodes = binary_reflected_graycodes(length=3)
for c, (o, s, p, r) in zip(count(), binary_reflected_graycodes_reduce(example_graycodes)):
    print('{}\t{}\t{}'.format(bin(c), bin(o), p))

0b0	0b0	0
0b1	0b1	1
0b10	0b11	0
0b11	0b10	2
0b100	0b110	0
0b101	0b111	1
0b110	0b101	0
0b111	0b100	2


### Direct generation

It is possible to write a function `graycodes_direct` which provides codes and changing-bit positions in parallel, here is the code:

In [36]:
python_code(graycodes_direct)

```python
def graycodes_direct(n):
    """
    Returns an iterable of Gray codes of length `n`, each paired with changing-bit positions.

    """
    code = 0

    def gen(i):
        nonlocal code
        if i > -1:
            yield from gen(i-1)
            code = toggle_bit(code, i)
            yield code, i
            yield from gen(i-1)
             

    yield code, 0
    yield from gen(n-1)

```

and can be used to build the same example table; observe that the changing-bit positions sequence is shifted *forward* by one:

In [38]:
for i, (code, changingbit_position) in zip(count(), graycodes_direct(3)):
    print('{}\t{}\t{}'.format(bin(i),bin(code), changingbit_position))

0b0	0b0	0
0b1	0b1	0
0b10	0b11	1
0b11	0b10	0
0b100	0b110	2
0b101	0b111	0
0b110	0b101	1
0b111	0b100	0


## *Unranking* Gray codes

Although not required, we can work the relation $k_{i−1} = k_i \oplus  g_{i-1}$ backwards, namely given a Gray code `g` find the corresponding position or rank `k`. To succeed, we think inductively as follows: rewrite the same schema, starting from the very left, namely the most significant part

$$
\begin{array}{ccccc}
    b_{n-1} & & b_{n-2} & & b_{n-3} \\
    = & \searrow & = & \searrow & = \\
    0 & & b_{n-1} & & b_{n-2} \\
    \oplus & & \oplus & & \oplus \\
    g_{n-1} & & g_{n-2} & & g_{n-3} \\
\end{array}
$$

where the row in the middle starts with 0 because it is shifted to the right by one. Doing the leftmost $\oplus$ and copying it according to the arrow -- after all, the middle row *is* the first one, just shifted -- we get:

$$
\begin{array}{ccccc}
    g_{n-1} & & b_{n-2} & & b_{n-3} \\
    = & \searrow & = & \searrow & = \\
    0 & & g_{n-1} & & b_{n-2} \\
    \oplus & & \oplus & & \oplus \\
    g_{n-1} & & g_{n-2} & & g_{n-3} \\
\end{array}
$$

We can complete the induction steps, obtaining:

$$
\begin{array}{ccccc}
    g_{n-1} & & g_{n-2}\oplus g_{n-1} & & g_{n-3}\oplus g_{n-2}\oplus g_{n-1} \\
    = & \searrow & = & \searrow & = \\
    0 & & g_{n-1} & & g_{n-2}\oplus g_{n-1} \\
    \oplus & & \oplus & & \oplus \\
    g_{n-1} & & g_{n-2} & & g_{n-3} \\
\end{array}
$$

it seems that we have to *cumulate* the given gray code. In order to derive working Python code, we can split the above schema according the following equivalent one:

$$
\begin{array}{ccc}
    b_{n-1} & b_{n-2} & b_{n-3} \\
    = & = & = \\
    g_{n-1} & g_{n-1} & g_{n-1} \\
     & \oplus & \oplus \\
      & g_{n-2} & g_{n-2} \\
     &  & \oplus \\
      &  & g_{n-3} \\
\end{array}
$$


This argument is implemented in function `gray_position`, where we code the repetition of each symbol $g_{i}$ building a sequence of ones of the correct length at each iteration, multiplied by a check that ensure that such symbol is actually 1:

In [40]:
python_code(graycode_rank)

```python
def graycode_rank(g):
    """
    Returns the *position* (namely *ranking*) of graycode `g` respect binary reflected construction.

    Examples
    ========

    >>> bin(graycode_rank(0b111010101))
    '0b101100110'

    """
    k=0
    for i in reversed(range(g.bit_length())):
        k ^= set_all(i+1) * is_on(g, i)

    return k

```

where auxiliary functions are defined as follows:

In [41]:
python_code(is_on, set_all)

```python
def is_on(S, j): 
    """
    Returns 1 if and only if the `j`-th item of the set `S` is on.

    Examples
    ========

    Check if the 3-th and then 2-nd item of the set is on:
    >>> S = 0b101010
    >>> is_on(S, 3), is_on(S, 2)
    (1, 0)

    """
    return (S & (1 << j)) >> j

def set_all(n):
    return (1 << n) - 1

```

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Gray codes tutorial</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/graycodes.ipynb" rel="dct:source">https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/graycodes.ipynb</a>.